In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

from train_evaluate import NMR_prediction
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop
from model_zoo.NMR_gcn import NMR_GCN
from create_graph_data import create_graph
from node_embeddings import create_node_embeddings
from create_adjaency_matrix_from_labeled_pdb import build_adjacency_matrix
from dgl import save_graphs, load_graphs
from tqdm import tqdm

In [2]:
num_test = 462
# 399

atom_name_dim=256

bound_orig_dim=32

atom_type_dim=32

ab_dim=64

dl_dim=64

pf_dim=64

mono_accurate_dim=128

mono_simple_dim=256

me_dim=32

ser_dim=32

s_dim=64

ac_dim=64

gc_dim=32


in_size = atom_name_dim + bound_orig_dim + atom_type_dim + ab_dim + dl_dim + \
            pf_dim + mono_accurate_dim + mono_simple_dim + me_dim + ser_dim + \
            s_dim + ac_dim + gc_dim

hidden_size_1 = int(in_size / 2)
hidden_size_2 = int(hidden_size_1 / 2)
hidden_size_3 = 256
hidden_size_4 = 128

num_epoch = 20000
lr = 1e-4

# num_epoch = 5000
# lr = 1e-4

threshold_carbon=1.65
threshold_hydrogen=1.18
threshold_general=1.5
threshold_interaction=5.0


seed=97211


data_dir = 'dataset/Godess_final_data/Godess_data_rev/'

additional_drop = [221, 325, 2304, 1184, 1408, 1984, 1976, 1506, 525, 1531, 746, 2000, 1053, 1328, 2114, 94, 929, 
                  2134, 2287, 1575, 501, 1034, 839, 646, 1602, 2403, 1857]

Create = create_graph(data_dir=data_dir, 
                      num_test=num_test, 
                      additional_drop=additional_drop, 
                      seed=seed)

In [3]:
g = dgl.DGLGraph()

train_test_indicator = Create.generate_train_test_split()

In [4]:
data_dir= 'dataset/Godess_final_data/Godess_data_rev/'
num_test = 462
additional_drop = [221, 325, 2304, 1184, 1408, 1984, 1976, 1506, 525, 1531, 
                   746, 2000, 1053, 1328, 2114, 94, 929, 2134, 2287, 1575, 
                   501, 1034, 839, 646, 1602, 2403, 1857]
seed = 97211

In [5]:
from GODDESS import GODESSDataset

In [6]:
GODESSDataset

GODDESS.GODESSDataset

In [7]:
# import dgl
# from dgl.data import DGLDataset
# import torch

# class GODESSDataset(DGLDataset):
#     def __init__(self):
#         super(GODESSDataset, self).__init__(name = 'GlycoNMR_GODESS')
    
#     def process(self):
#         # Implement the processing logic to generate your graphs
#         self.graphs = []
        
#         self.train_test_indicator = []
        
#         self.glycan_name = []
        
#         C = create_graph(data_dir=data_dir, 
#                          num_test=num_test, 
#                          additional_drop=additional_drop, 
#                          seed=seed)
        
#         train_test_indicator = C.generate_train_test_split()
        
#         for i in tqdm(range(len(C.files_labels_list))):
#             f = C.files_labels_list[i]
#             if train_test_indicator[i] == 1:
#                 temp_g = C.create_single_graph(f, in_train_set=False, in_test_set=True)
#                 self.train_test_indicator.append(1)
#                 self.glycan_name.append(f)
#             elif train_test_indicator[i] == 0:
#                 temp_g = C.create_single_graph(f, in_train_set=True, in_test_set=False)
#                 self.train_test_indicator.append(0)
#                 self.glycan_name.append(f)
                
                
#             elif train_test_indicator[i] == -1:
#                 print(f, 'is dropped due to rare monosaccharide appearance')
#                 continue
                
#             self.graphs.append(temp_g)
        
#     def __getitem__(self, idx):
#         return self.graphs[idx], self.train_test_indicator[idx], self.glycan_name[idx]
#     def __len__(self):
#         return len(self.graphs)


In [8]:
dataset = GODESSDataset()

  0%|▏                                        | 10/2399 [00:00<00:31, 76.77it/s]

labeled_all_325.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_2304.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_1184.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_1408.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_1984.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_1976.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_1506.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_525.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_1531.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_746.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_2000.pdb.csv is dropped due to rare monosaccharide appearance


  1%|▍                                        | 29/2399 [00:00<00:27, 85.09it/s]

labeled_all_1053.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_1328.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_2114.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_94.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_929.pdb.csv is dropped due to rare monosaccharide appearance
labeled_all_2403.pdb.csv is dropped due to rare monosaccharide appearance


  3%|█▏                                       | 72/2399 [00:01<00:45, 51.64it/s]

...

labeled_all_2241.pdb.csv is dropped due to rare monosaccharide appearance


 99%|██████████████████████████████████████▌| 2369/2399 [00:49<00:00, 56.48it/s]

labeled_all_2334.pdb.csv is dropped due to rare monosaccharide appearance


100%|███████████████████████████████████████| 2399/2399 [00:49<00:00, 48.26it/s]

labeled_all_2284.pdb.csv is dropped due to rare monosaccharide appearance


In [12]:
g1, train_test_id, g1_name = dataset[1]

In [13]:
g1

Graph(num_nodes=99, num_edges=303,
      ndata_schemes={'feat': Scheme(shape=(1120,), dtype=torch.float32), 'shift_value': Scheme(shape=(), dtype=torch.float32), 'Carbon_Hydrogen_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'train_carbon_mask': Scheme(shape=(), dtype=torch.bool), 'test_carbon_mask': Scheme(shape=(), dtype=torch.bool), 'train_hydrogen_mask': Scheme(shape=(), dtype=torch.bool), 'test_hydrogen_mask': Scheme(shape=(), dtype=torch.bool), 'coordinate': Scheme(shape=(3,), dtype=torch.float64)}
      edata_schemes={})

In [ ]:
import torch_geometric